# downloading data form github  repo

In [1]:
from pydriller import Repository

repo_url = "spring-framework"

for commit in Repository(repo_url).traverse_commits():
    print(commit.__dict__)   
    break   


{'_c_object': <git.Commit "8119659fb1e4ae6fabe8897c42ba7629fda07b21">, '_conf': <pydriller.utils.conf.Conf object at 0x0000022A54B06C80>, '_stats_cache': None}


In [2]:
# to download repo wirte in the terminal git clone "github link"

In [3]:
import csv
'''
with open("fixed_file.csv", "w", newline="", encoding="utf-8") as f:
    df = csv.writer(f)
    df.writerow(["Hash", "Message", "Type", "Old_Code", "New_Code"])  

    for commit in Repository(repo_url).traverse_commits():
        if "fix" in commit.msg.lower():
            for m in commit.modified_files:
                if m.filename.endswith(".java"):
                    df.writerow([
                        commit.hash,
                        commit.msg,
                        m.change_type.name,
                        m.source_code_before, 
                        m.source_code 
                    ])
'''                    




'\nwith open("fixed_file.csv", "w", newline="", encoding="utf-8") as f:\n    df = csv.writer(f)\n    df.writerow(["Hash", "Message", "Type", "Old_Code", "New_Code"])  \n\n    for commit in Repository(repo_url).traverse_commits():\n        if "fix" in commit.msg.lower():\n            for m in commit.modified_files:\n                if m.filename.endswith(".java"):\n                    df.writerow([\n                        commit.hash,\n                        commit.msg,\n                        m.change_type.name,\n                        m.source_code_before, \n                        m.source_code \n                    ])\n'

### function to download commits from repo

In [4]:
import os
def extract_fixes(repo_path, output_file="fixed_file.csv"):
  
    file_exists = os.path.isfile(output_file)

    with open(output_file, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Hash", "Message", "Type", "Old_Code", "New_Code"])
        
        for commit in Repository(repo_path).traverse_commits():
            if "fix" in commit.msg.lower():
                for m in commit.modified_files:
                    if m.filename.endswith(".java"): #just java files with fix commits
                        writer.writerow([
                            commit.hash,
                            commit.msg,
                            m.change_type.name,
                            m.source_code_before or "",
                            m.source_code or ""
                        ])



In [5]:
#extract_fixes("defects4j")

In [6]:
#extract_fixes("accumulo")

In [7]:
#extract_fixes("camel")

In [8]:
#extract_fixes("ISSRE2023-BugSeverityPrediction")

In [9]:
#extract_fixes("wicket")

In [10]:
#extract_fixes("jackrabbit-oak")

In [ ]:
#extract_fixes("maven")

# clean the data

In [12]:
import pandas as pd  
df=pd.read_csv("fixed_file.csv")
df

,Hash,Message,Type,Old_Code,New_Code
0,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
1,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,ADD,NaN,package org.springframework.http.client;\n\nim...
2,ffc785471bbe579aaef282720baef00a44d46435,Fix checkstyle error,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
3,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
4,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
...,...,...,...,...,...
26757,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
26758,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
26759,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
26760,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...


### remove the messages from the code

In [13]:
import re
def remove_comments(code):
    if not isinstance(code, str): 
        return code
    code=re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)  
    code=re.sub(r'//.*', '', code) 
    return code.strip()


In [14]:
df[["Old_Code", "New_Code"]] = df[["Old_Code", "New_Code"]].applymap(remove_comments)

C:\Users\hp\AppData\Local\Temp\ipykernel_21140\2932069089.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["Old_Code", "New_Code"]] = df[["Old_Code", "New_Code"]].applymap(remove_comments)


In [15]:
print(df["New_Code"].iloc[0])

package org.springframework.http.client;

import java.io.FilterInputStream;
import java.io.IOException;
import java.io.InputStream;
import java.io.UncheckedIOException;
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.net.http.HttpTimeoutException;
import java.nio.ByteBuffer;
import java.time.Duration;
import java.util.Collections;
import java.util.Locale;
import java.util.Set;
import java.util.TreeSet;
import java.util.concurrent.CancellationException;
import java.util.concurrent.CompletableFuture;
import java.util.concurrent.ExecutionException;
import java.util.concurrent.Executor;
import java.util.concurrent.Flow;
import java.util.concurrent.TimeUnit;
import java.util.concurrent.atomic.AtomicBoolean;

import org.springframework.http.HttpHeaders;
import org.springframework.http.HttpMethod;
import org.springframework.lang.Nullable;
import org.springframework.util.StreamUtils;
import org.springframewo

In [16]:
print(df["Type"].unique())

['MODIFY' 'ADD' 'RENAME' 'DELETE']


In [17]:
a=['MODIFY', 'ADD', 'RENAME', 'DELETE']
for t in a:
    print(f"{t}: {len(df[df['Type'] == t])}")


MODIFY: 24066
ADD: 1695
RENAME: 615
DELETE: 386


### take the only row that have modify in the type

In [18]:
df=df[df["Type"]=="MODIFY"]


In [19]:
df=df.reset_index(drop=True)
df

,Hash,Message,Type,Old_Code,New_Code
0,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,MODIFY,package org.springframework.http.client;\n\nim...,package org.springframework.http.client;\n\nim...
1,ffc785471bbe579aaef282720baef00a44d46435,Fix checkstyle error,MODIFY,package org.springframework.http.client;\n\nim...,package org.springframework.http.client;\n\nim...
2,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
3,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
4,81b4020fc6392025e5e55b2d26b7cd5bac1ca7fe,Do not load concrete types in annotation metad...,MODIFY,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
...,...,...,...,...,...
24061,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,package org.apache.maven.cling.executor.intern...,package org.apache.maven.cling.executor.intern...
24062,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,package org.apache.maven.cling.executor.impl;\...,package org.apache.maven.cling.executor.impl;\...
24063,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,package org.apache.maven.impl.standalone;\n\ni...,package org.apache.maven.impl.standalone;\n\ni...
24064,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,MODIFY,package org.apache.maven.api.plugin.testing.st...,package org.apache.maven.api.plugin.testing.st...


In [20]:
df=df.drop('Type', axis=1)
df

,Hash,Message,Old_Code,New_Code
0,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,package org.springframework.http.client;\n\nim...,package org.springframework.http.client;\n\nim...
1,ffc785471bbe579aaef282720baef00a44d46435,Fix checkstyle error,package org.springframework.http.client;\n\nim...,package org.springframework.http.client;\n\nim...
2,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
3,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
4,81b4020fc6392025e5e55b2d26b7cd5bac1ca7fe,Do not load concrete types in annotation metad...,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
...,...,...,...,...
24061,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,package org.apache.maven.cling.executor.intern...,package org.apache.maven.cling.executor.intern...
24062,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,package org.apache.maven.cling.executor.impl;\...,package org.apache.maven.cling.executor.impl;\...
24063,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,package org.apache.maven.impl.standalone;\n\ni...,package org.apache.maven.impl.standalone;\n\ni...
24064,b4ee1aa0450ff87433a978c2989c0ac432496863,Resolver 2.0.11 (#11043)\n\nUpdate to Resolver...,package org.apache.maven.api.plugin.testing.st...,package org.apache.maven.api.plugin.testing.st...


In [21]:
print(df['Message'].iloc[2])

Fix annotation arrays support in ClassFile metadata

As of gh-33616, Spring now supports metadata reading with the ClassFile
API on JDK 24+ runtimes. This commit fixes a bug where
`ArrayStoreException` were thrown when reading annotation attribute
values for arrays.

Fixes gh-35252


In [22]:
keywords = ['bug', 'error', 'issue', 'fault', 'fail', 'exception']
for word in keywords:
    print(df["Message"].str.contains(word, case=False).sum())

2544
3513
2959
1031
1105
1116


In [23]:
df.columns

Index(['Hash', 'Message', 'Old_Code', 'New_Code'], dtype='object')

### search for the lines that have error, bugs,fault,fail,exception or isssue in their message 

In [24]:
df_target=pd.DataFrame(columns=['Hash', 'Message', 'Old_Code', 'New_Code'])
for word in keywords:
    df_sub=df[df["Message"].str.contains(word,case=False)]
    df_target=pd.concat([df_target,df_sub],ignore_index=True)
df_target    

,Hash,Message,Old_Code,New_Code
0,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
1,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
2,764336f0f201c34ba0e636324e4b31da922c81c1,Fix Jetty DataBufferFactory memory leak\n\nPri...,package org.springframework.core.io.buffer;\n\...,package org.springframework.core.io.buffer;\n\...
3,764336f0f201c34ba0e636324e4b31da922c81c1,Fix Jetty DataBufferFactory memory leak\n\nPri...,package org.springframework.core.io.buffer;\n\...,package org.springframework.core.io.buffer;\n\...
4,764336f0f201c34ba0e636324e4b31da922c81c1,Fix Jetty DataBufferFactory memory leak\n\nPri...,package org.springframework.http.server.reacti...,package org.springframework.http.server.reacti...
...,...,...,...,...
12263,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12264,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12265,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\n\npublic ...,package org.apache.maven.project;\n\n\n@Deprec...
12266,84bf592feac1aa409a04b4fa69d0dee61ec0cf7a,Fix recursive update issue in SmartProjectComp...,package org.apache.maven.lifecycle.internal.bu...,package org.apache.maven.lifecycle.internal.bu...


In [27]:
print(df_target["Message"].iloc[12267])

Fix recursive update issue in SmartProjectComparator (#10997)

Fixes #10995

The SmartProjectComparator.getProjectWeight() method was using
ConcurrentHashMap.computeIfAbsent() in a recursive context, which
could lead to IllegalStateException: Recursive update when calculating
project weights in complex dependency graphs or concurrent scenarios.

Changes:
- Replace computeIfAbsent() with explicit get() + putIfAbsent() pattern
- Eliminate recursive calls to computeIfAbsent() that violate
  ConcurrentHashMap's internal constraints
- Maintain thread safety using putIfAbsent() for concurrent access
- Add comprehensive test case to verify the fix under concurrent load

The fix preserves all existing functionality while eliminating the
recursive update exception that could occur during parallel builds
of large multi-module projects.


In [26]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12268 entries, 0 to 12267
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Hash      12268 non-null  object
 1   Message   12268 non-null  object
 2   Old_Code  12267 non-null  object
 3   New_Code  12268 non-null  object
dtypes: object(4)
memory usage: 383.5+ KB


In [28]:
print(df_target.iloc[1])

Hash                 2b7f88ee449fb37c81f2e3649beaa643f9537357
Message     Fix annotation arrays support in ClassFile met...
Old_Code    package org.springframework.core.type;\n\nimpo...
New_Code    package org.springframework.core.type;\n\nimpo...
Name: 1, dtype: object


In [29]:
df_target[df_target.duplicated(subset=['Old_Code', 'New_Code'], keep=False)]


,Hash,Message,Old_Code,New_Code
0,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
1,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
5,8830c99effac591d19be25de2f185928dd73ba08,Merge bug mining into master (#135)\n\n* sligh...,package edu.washington.cs.mut.testrunner;\n\ni...,package edu.washington.cs.mut.testrunner;\n\ni...
6,8830c99effac591d19be25de2f185928dd73ba08,Merge bug mining into master (#135)\n\n* sligh...,package edu.washington.cs.mut.testrunner;\n\ni...,package edu.washington.cs.mut.testrunner;\n\ni...
67,48e1cc0c1c3ac7c9d8689f743b7d2093d08a0bcd,ACCUMULO-422 Fixed two bugs caused by tserver ...,package org.apache.accumulo.server.master.tabl...,package org.apache.accumulo.server.master.tabl...
...,...,...,...,...
12263,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12264,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12265,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\n\npublic ...,package org.apache.maven.project;\n\n\n@Deprec...
12266,84bf592feac1aa409a04b4fa69d0dee61ec0cf7a,Fix recursive update issue in SmartProjectComp...,package org.apache.maven.lifecycle.internal.bu...,package org.apache.maven.lifecycle.internal.bu...


In [30]:
df_target[df_target.duplicated(subset=['New_Code'], keep=False)]


,Hash,Message,Old_Code,New_Code
0,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
1,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
5,8830c99effac591d19be25de2f185928dd73ba08,Merge bug mining into master (#135)\n\n* sligh...,package edu.washington.cs.mut.testrunner;\n\ni...,package edu.washington.cs.mut.testrunner;\n\ni...
6,8830c99effac591d19be25de2f185928dd73ba08,Merge bug mining into master (#135)\n\n* sligh...,package edu.washington.cs.mut.testrunner;\n\ni...,package edu.washington.cs.mut.testrunner;\n\ni...
67,48e1cc0c1c3ac7c9d8689f743b7d2093d08a0bcd,ACCUMULO-422 Fixed two bugs caused by tserver ...,package org.apache.accumulo.server.master.tabl...,package org.apache.accumulo.server.master.tabl...
...,...,...,...,...
12263,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12264,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12265,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\n\npublic ...,package org.apache.maven.project;\n\n\n@Deprec...
12266,84bf592feac1aa409a04b4fa69d0dee61ec0cf7a,Fix recursive update issue in SmartProjectComp...,package org.apache.maven.lifecycle.internal.bu...,package org.apache.maven.lifecycle.internal.bu...


In [31]:
df_target[df_target.duplicated(subset=['Old_Code'], keep=False)]


,Hash,Message,Old_Code,New_Code
0,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
1,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
5,8830c99effac591d19be25de2f185928dd73ba08,Merge bug mining into master (#135)\n\n* sligh...,package edu.washington.cs.mut.testrunner;\n\ni...,package edu.washington.cs.mut.testrunner;\n\ni...
6,8830c99effac591d19be25de2f185928dd73ba08,Merge bug mining into master (#135)\n\n* sligh...,package edu.washington.cs.mut.testrunner;\n\ni...,package edu.washington.cs.mut.testrunner;\n\ni...
10,9d13f546ebb1e716c1d25c35225190b1b2805975,ACCUMULO-155 moved some user iterators into th...,package org.apache.accumulo.core.client;\n\nim...,package org.apache.accumulo.core.client;\n\nim...
...,...,...,...,...
12263,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12264,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\nimport ja...,package org.apache.maven.project;\n\nimport ja...
12265,8ce8cd0ccc8b646f231d1e5e7da54f5dcfd7f6cc,Deprecate old project builder API in favor of ...,package org.apache.maven.project;\n\n\npublic ...,package org.apache.maven.project;\n\n\n@Deprec...
12266,84bf592feac1aa409a04b4fa69d0dee61ec0cf7a,Fix recursive update issue in SmartProjectComp...,package org.apache.maven.lifecycle.internal.bu...,package org.apache.maven.lifecycle.internal.bu...


### search for the row that have the same old and new code

In [34]:
df_false=df_target[df_target['Old_Code']==df_target["New_Code"]]
print(len(df_false))
df_false

876


,Hash,Message,Old_Code,New_Code
10,9d13f546ebb1e716c1d25c35225190b1b2805975,ACCUMULO-155 moved some user iterators into th...,package org.apache.accumulo.core.client;\n\nim...,package org.apache.accumulo.core.client;\n\nim...
12,9d13f546ebb1e716c1d25c35225190b1b2805975,ACCUMULO-155 moved some user iterators into th...,package org.apache.accumulo.core.client.impl;\...,package org.apache.accumulo.core.client.impl;\...
17,9d13f546ebb1e716c1d25c35225190b1b2805975,ACCUMULO-155 moved some user iterators into th...,package org.apache.accumulo.core.iterators;\n\...,package org.apache.accumulo.core.iterators;\n\...
30,9d13f546ebb1e716c1d25c35225190b1b2805975,ACCUMULO-155 moved some user iterators into th...,package org.apache.accumulo.core.iterators.fil...,package org.apache.accumulo.core.iterators.fil...
141,6e526d702caaae157c44254b041497d18af761d0,ACCUMULO-1129 - Implemented new getConnector m...,package org.apache.accumulo.core.security.thri...,package org.apache.accumulo.core.security.thri...
...,...,...,...,...
12075,d98b1842d5e5fa842f2aa21119adbdd4080cdc3d,Fix minor typos\n\nFix various minor typos in ...,package org.apache.maven.plugin.coreit;\n\n\n\...,package org.apache.maven.plugin.coreit;\n\n\n\...
12076,d98b1842d5e5fa842f2aa21119adbdd4080cdc3d,Fix minor typos\n\nFix various minor typos in ...,package org.apache.maven.plugin.coreit;\n\n\n\...,package org.apache.maven.plugin.coreit;\n\n\n\...
12188,bcd5d9c9f9ef87bb103dcabb24be62c7128d2fff,DI improvements (#1717)\n\n* Code cleanup\r\n*...,package org.apache.maven.api.di;\n\nimport jav...,package org.apache.maven.api.di;\n\nimport jav...
12189,bcd5d9c9f9ef87bb103dcabb24be62c7128d2fff,DI improvements (#1717)\n\n* Code cleanup\r\n*...,package org.apache.maven.api.di;\n\nimport jav...,package org.apache.maven.api.di;\n\nimport jav...


### delete the row that have the same old and new code

In [37]:
df_target=df_target[df_target['Old_Code']!=df_target['New_Code']]
df_target=df_target.dropna()
df_target.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11391 entries, 0 to 12267
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Hash      11391 non-null  object
 1   Message   11391 non-null  object
 2   Old_Code  11391 non-null  object
 3   New_Code  11391 non-null  object
dtypes: object(4)
memory usage: 445.0+ KB


In [49]:
print(df_target["Old_Code"].iloc[0])

package org.springframework.core.type.classreading;


import java.lang.classfile.Annotation;
import java.lang.classfile.AnnotationElement;
import java.lang.classfile.AnnotationValue;
import java.lang.classfile.attribute.RuntimeVisibleAnnotationsAttribute;
import java.lang.constant.ClassDesc;
import java.lang.reflect.Array;
import java.util.Collections;
import java.util.LinkedHashMap;
import java.util.Map;
import java.util.Objects;
import java.util.Set;
import java.util.stream.Collectors;
import java.util.stream.Stream;

import org.jspecify.annotations.Nullable;

import org.springframework.core.annotation.AnnotationFilter;
import org.springframework.core.annotation.MergedAnnotation;
import org.springframework.core.annotation.MergedAnnotations;
import org.springframework.util.ClassUtils;


abstract class ClassFileAnnotationMetadata {

	static MergedAnnotations createMergedAnnotations(String className, RuntimeVisibleAnnotationsAttribute annotationAttribute, @Nullable ClassLoader classLoad

### function to compare old and new code

In [42]:
import difflib
def compare_code(old_code, new_code):
    old_lines=str(old_code).splitlines()
    new_lines=str(new_code).splitlines()
    diff=difflib.unified_diff(
        old_lines, new_lines,
        fromfile="Old_Code", 
        tofile="New_Code", 
        lineterm=""
    )
    return "\n".join(diff)


In [58]:
print(compare_code(df_target["Old_Code"].iloc[10010],df_target["New_Code"].iloc[10010]))
print(df_target["Message"].iloc[10010])

--- Old_Code
+++ New_Code
@@ -1,18 +1,18 @@
 package org.apache.jackrabbit.oak.spi.state;
 
+import javax.annotation.Nonnull;
+
 import org.apache.jackrabbit.oak.api.CommitFailedException;
-
-import javax.annotation.Nonnull;
 
 public interface NodeStoreBranch {
 
     
     @Nonnull
-    NodeState getRoot();
+    NodeState getBase();
 
     
     @Nonnull
-    NodeState getBase();
+    NodeState getRoot();
 
     
     void setRoot(NodeState newRoot);
OAK-251 : CommitEditor is global to ContentRepository instance  (WIP)
- add initial draft that allows to extend the commit-editor passed in to the repository constructor

OAK-51 : Implement JCR Access Control Management 
- enforce dummy permission validation upon commit

OAK-50 : UserManagement
- enforce user-validation upon commit

OAK-64 : Privilege Management 
- enforce privilege validation upon commit (currently disabled due to core-tests that don't setup mandatory
   repository content and therefore fail).
   
OAK-125 	: Improved na

In [60]:
df_bug=pd.DataFrame(columns=["AfterFix"])
df_bug["AfterFix"] = df_target["Message"].str.extract(r'(?i)fix[:\s]*(.*)')
df_bug

,AfterFix
0,annotation arrays support in ClassFile metadata
1,annotation arrays support in ClassFile metadata
2,Jetty DataBufferFactory memory leak
3,Jetty DataBufferFactory memory leak
4,Jetty DataBufferFactory memory leak
...,...
12263,es apache/maven#10984
12264,es apache/maven#10984
12265,es apache/maven#10984
12266,recursive update issue in SmartProjectComparat...


In [69]:
df_bug=df_bug.reset_index()

In [72]:
df_target["Message"].iloc[11390]

"Fix recursive update issue in SmartProjectComparator (#10997)\n\nFixes #10995\n\nThe SmartProjectComparator.getProjectWeight() method was using\nConcurrentHashMap.computeIfAbsent() in a recursive context, which\ncould lead to IllegalStateException: Recursive update when calculating\nproject weights in complex dependency graphs or concurrent scenarios.\n\nChanges:\n- Replace computeIfAbsent() with explicit get() + putIfAbsent() pattern\n- Eliminate recursive calls to computeIfAbsent() that violate\n  ConcurrentHashMap's internal constraints\n- Maintain thread safety using putIfAbsent() for concurrent access\n- Add comprehensive test case to verify the fix under concurrent load\n\nThe fix preserves all existing functionality while eliminating the\nrecursive update exception that could occur during parallel builds\nof large multi-module projects."

In [68]:
df_target=df_target.reset_index(drop=True)

In [ ]:
count=df_bug["AfterFix"].str.contains("apache/maven#10984", na=False).sum()
print(count)


28


In [74]:
print(df_bug["AfterFix"].nunique())

1848
